In [29]:
import requests, os, binascii, hashlib

# Grab English Wordlist From BIP Git Repo

In [30]:
english_wordlist_url = "https://raw.githubusercontent.com/bitcoin/bips/master/bip-0039/english.txt"
response = requests.get(english_wordlist_url)
wordlist = response.text.splitlines()
assert len(wordlist) == 2048

# Produce Mnemonic From Random Bytes

In [31]:
def generate_entropy(n):
    return os.urandom(n)

def mnemonic_from_entropy(entropy_bytes):
    entropy_bits_length = len(entropy_bytes) * 8
    checksum_bits_length = entropy_bits_length // 32
    sequence_length = (entropy_bits_length + checksum_bits_length) // 11
    
    # Prepare entropy bits
    entropy_bits = bin(int(binascii.hexlify(entropy), 16))[2:]
    padded_entropy_bits = entropy_bits.zfill(entropy_bits_length)
    
    # Prepare checksum
    entropy_hash = hashlib.sha256(entropy).hexdigest()
    entropy_hash_int = int(entropy_hash, 16)
    entropy_hash_bits = bin(int(entropy_hash, 16))[2:]
    padded_entropy_hash_bits = entropy_hash_bits.zfill(256)
    checksum = padded_entropy_hash_bits[:checksum_bits_length]
    
    # Take 11 bit slices of padded_entropy_bits + checksum
    # Interpret as int, pluck work from wordlist at this index
    sequence = padded_entropy_bits + checksum
    result = []
    for i in range(sequence_length):
        index = int(sequence[i * 11:(i + 1) * 11], 2)
        result.append(wordlist[index])
    return result

In [77]:
entropy = generate_entropy(32)
entropy

b'F\xfa\x02\xdb\xe6\x03\xd6\x9e\n\xefW\x7f+\x95\xc0\xa5\xd1b\xca\xe9\xf4\xd1\x85\x106\xb2v\xaba\xe7\x11\xee'

In [74]:
mnemonic_from_entropy(entropy)

['belt',
 'shell',
 'fossil',
 'bullet',
 'impose',
 'monkey',
 'common',
 'lunch',
 'tone',
 'season',
 'boil',
 'share',
 'shop',
 'angle',
 'humor',
 'adapt',
 'path',
 'plug',
 'movie',
 'metal',
 'trend',
 'bracket',
 'dilemma',
 'depth']